In [1]:
from tensorflow import keras
import tensorflow as tf
from keras.layers import *
from keras.models import Model
from keras.applications import ResNet50



def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(num_filters, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    return x

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding='same')(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)

    return x

def build_resnet50_unet(input_shape):
    """input"""
    inputs = Input(input_shape)

    """Pre-trained ResNet50"""
    resnet = ResNet50(include_top=False, weights='imagenet', input_tensor=inputs)

    """Encoder"""
    # shapes:s
    s1 = resnet.get_layer('input_1').output             ## 512
    s2 = resnet.get_layer('conv1_relu').output          ## 256
    s3 = resnet.get_layer('conv2_block3_out').output    ## 128
    s4 = resnet.get_layer('conv3_block4_out').output    ## 64

    """Bridge"""
    b1 = resnet.get_layer('conv4_block6_out').output    ## 32

    """Decoder"""
    d1 = decoder_block(b1, s4, 512)                     ## 64
    d2 = decoder_block(d1, s3, 512)                     ## 128
    d3 = decoder_block(d2, s2, 512)                     ## 256
    d4 = decoder_block(d3, s1, 512)                     ## 512

    """Output"""
    outputs = Conv2D(1, 1, padding='same', activation='sigmoid')(d4)

    model = Model(inputs, outputs)
    return model



if __name__ == "__main__":
    input_shape = (512, 512, 3)
    model = build_resnet50_unet(input_shape)
    model.summary()
    model.save('512_resnet50_unet.h5')

tf.keras.backend.clear_session()

2022-06-18 14:14:44.021260: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-18 14:14:44.021288: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-06-18 14:14:45.239310: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-18 14:14:45.239345: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-18 14:14:45.239359: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Luise): /proc/driver/nvidia/version does not exist
2022-06-18 14:14:45.239513: I tensorflow/core/platform/cpu_feature_guar

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              